<img height="40px" src="https://science-tokyo.box.com/shared/static/qjfkgaerixyqzkh91r93fnbett6ea0ox.png">

In [ ]:
# Preparation for auto-grading
unit = "AI6"
!pip install otter-grader 
!wget https://github.com/keisuke-yanagisawa/isct-efdsai/raw/refs/heads/main/supplementary_materials/{unit}/tests.tar.gz -O {unit}.tar.gz
!tar -zxvf {unit}.tar.gz 
!rm {unit}.tar.gz
import otter
grader = otter.Notebook()

# 基盤人工知能演習 第6回 ニューラルネットワークによる多クラス分類 補習資料

# Supplemental Material: Exercises in Fundamentals of Data Science (6) - 	AI6: Multi-class Classification with Neural Networks

## 1 | 手書き数字を用いた多クラス分類

## 1 | Multi-class Classification using Handwritten Digits

#### 事前準備

#### Preparation

In [ ]:
%pip install livelossplot

In [ ]:
# load digits dataset

from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

X, y = load_digits(return_X_y=True)

# train: 80%, val: 10%, test: 10%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/10, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=1/9, random_state=42)

In [ ]:
# scale data using StandardScaler (z-score normalization) to stabilize learning rates
# scaling is done on the training set, and the same scaling is applied to the validation and test sets
# this is to prevent data leakage (i.e., information from the validation or test sets being used to scale the training set)
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [ ]:
import torch

X_train_torch = torch.tensor(X_train, dtype=torch.float)
y_train_torch = torch.tensor(y_train, dtype=torch.long)
train_dataset = torch.utils.data.TensorDataset(X_train_torch, y_train_torch)

X_val_torch = torch.tensor(X_val, dtype=torch.float)
y_val_torch = torch.tensor(y_val, dtype=torch.long)
val_dataset = torch.utils.data.TensorDataset(X_val_torch, y_val_torch)

X_test_torch = torch.tensor(X_test, dtype=torch.float)
y_test_torch = torch.tensor(y_test, dtype=torch.long)
test_dataset = torch.utils.data.TensorDataset(X_test_torch, y_test_torch)


In [ ]:
# Create data loaders
batch_size = 16
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_loader  = torch.utils.data.DataLoader(test_dataset,  batch_size=batch_size, shuffle=True)

In [ ]:
def update_model(model, loss_fn, opt, train_loader):
  train_loss = 0
  train_correct = 0
  train_count = len(train_loader.dataset)
  
  for X, y in train_loader:
    y_pred = model(X) 
    
    _, predicted = torch.max(y_pred.data, 1) 
    train_correct += (predicted == y).sum().item() 
    
    loss = loss_fn(y_pred, y)        
    train_loss += loss.item()*len(y)
    
    opt.zero_grad()
    loss.backward()
    opt.step()
    
  mean_train_loss = train_loss / train_count
  train_accuracy = train_correct / train_count
  
  return mean_train_loss, train_accuracy    

In [ ]:
def evaluate_model(model, loss_fn, dataloader):
  model.eval()

  valid_loss = 0
  valid_correct = 0
  valid_count = len(dataloader.dataset)

  for X, y in dataloader:
    y_pred = model(X)
    
    _, predicted = torch.max(y_pred.data, 1) 
    valid_correct += (predicted == y).sum().item()
    
    loss = loss_fn(y_pred, y)
    valid_loss += loss.item()*len(y)
    
  mean_valid_loss = valid_loss / valid_count
  valid_accuracy = valid_correct / valid_count

  model.train()
  return mean_valid_loss, valid_accuracy

In [ ]:
from livelossplot import PlotLosses
def train(model, loss_fn, opt, train_loader, valid_loader, epoch=50, verbose=True):
  if verbose:
    liveloss = PlotLosses() # Initialize the drawing
  for i in range(epoch):
    train_loss, train_accuracy = update_model(model, loss_fn, opt, train_loader)
    valid_loss, valid_accuracy = evaluate_model(model, loss_fn, valid_loader)

    if verbose:
      liveloss.update({
          'loss': train_loss,
          'val_loss': valid_loss,
          'accuracy': train_accuracy,
          'val_accuracy': valid_accuracy,
      })
      liveloss.draw()  
  if verbose:
    print('Accuracy: {:.4f} (valid), {:.4f} (train)'.format(valid_accuracy, train_accuracy))
  return model # Return the trained model

#### 練習問題 1-1 | 交差エントロピー誤差の計算


#### Exercise 1-1 | Calculation of Cross-Entropy Loss


ある予測モデルに対してあるデータを入力したところ、予測結果 `q11_y_pred = (0.0, 0.0, 0.1, 0.5, 0.0, 0.2, 0.0, 0.1, 0.1, 0.0)` が得られた。正解ラベルが `q11_y_true = 3` （対応する予測確率は `0.5` ）であった時の交差エントロピー誤差`q11_loss`を算出せよ。

For a given prediction model with input data, the prediction result `q11_y_pred = (0.0, 0.0, 0.1, 0.5, 0.0, 0.2, 0.0, 0.1, 0.1, 0.0)` was obtained. Calculate the cross-entropy loss `q11_loss` when the true label is `q11_y_true = 3` (with the corresponding predicted probability of `0.5`).

In [ ]:

q11_y_pred = (0.0, 0.0, 0.1, 0.5, 0.0, 0.2, 0.0, 0.1, 0.1, 0.0)
q11_y_true = 3


In [ ]:
from torch import nn
import torch
loss=nn.CrossEntropyLoss()
q11_loss= ...
q11_loss

In [ ]:
grader.check("Exercise 1-1")

#### 練習問題 1-2 | 予測モデル構築の練習

#### Exercise 1-2 | Practice Building a Prediction Model

以下のコードを実行し、予測モデルを構築せよ。なおこの問題は穴埋め形式ではないので、自動添削対象外である。

Run the following code to build a prediction model. Note that this problem is not in fill-in-the-blank format, so it is not subject to automatic grading.

In [ ]:
torch.manual_seed(42)

q12_slp = torch.nn.Sequential(
  torch.nn.Linear(8*8, 10),
)

# Prepare loss functions and optimization methods
q12_loss_fn = torch.nn.CrossEntropyLoss()
q12_optimizer = torch.optim.SGD(q12_slp.parameters(), lr=0.02)

q12_trained_model = train(
  q12_slp, 
  q12_loss_fn, 
  q12_optimizer, 
  train_loader, 
  val_loader,
  epoch=15,
)

#### 練習問題 1-3 | テストデータの予測(1)

#### Exercise 1-3 | Prediction on Test Data (1)

`evaluate_model()` 関数を用いて `test_loader` に含まれる全てのテストデータについて予測し、平均 loss および正解率を算出せよ。

Using the `evaluate_model()` function, predict all test data contained in `test_loader` and calculate the average loss and accuracy.

In [ ]:
# 正答例を記述するセル
# 下記テストケースについて、テストが実行できるように各変数を定めるようにしてください。問題文に変数名を追加しても構いません。
# 結果を安定させるために、 torch.manual_seed(0) を最初に実行するようにしてください。
import torch
torch.manual_seed(0)
q13_loss, q13_acc = ...

In [ ]:
print(q13_loss,q13_acc)

In [ ]:
grader.check("Exercise 1-3")

#### 練習問題 1-4 | テストデータの予測(2)

#### Exercise 1-4 | Prediction on Test Data (2)

以下のコードは、入力に対してモデルが出力する確率を計算する関数 `predict_proba()` である。先ほど訓練したモデル `q12_trained_model` を使ってテストデータの0件目の手書き文字である `q14_X_test` を予測した時に、最も予測確率の高い数字と、その確率を算出せよ。

The following code is the function `predict_proba()` which calculates the probabilities that the model outputs for the input. Use the trained model `q12_trained_model` to predict the first handwritten digit from the test data `q14_X_test`, and calculate the digit with the highest predicted probability and that probability.

In [ ]:
def predict_proba(model, X):
  model.eval()
  with torch.no_grad():
    y_pred = model(X).softmax(dim=1)
  model.train()
  return y_pred

In [ ]:
q14_X_test = X_test_torch[:1] # Get only the first test data, which becomes (1, 64) torch.tensor
q14_X_test_copy = X_test_torch[:1] # for auto grading purpose

In [ ]:
... 
q14_max_num = ...
...
q14_max_prob = ...

print(q14_max_num, q14_max_prob)

In [ ]:
grader.check("Exercise 1-4")

#### 練習問題 1-5 | 多層パーセプトロンの構築

#### Exercise 1-5 | Building a Multi-layer Perceptron

`torch.nn.Linear(8*8, 32)`, `torch.nn.Sigmoid()`, `torch.nn.Linear(32, 10)` を直列につないだ `q15_mlp` を作成し、学習を行え。

Create `q15_mlp` by connecting `torch.nn.Linear(8*8, 32)`, `torch.nn.Sigmoid()`, and `torch.nn.Linear(32, 10)` in series, and perform training.

In [ ]:
torch.manual_seed(0)

q15_mlp = ...

# Prepare loss functions and optimization methods
q15_loss_fn = torch.nn.CrossEntropyLoss()
q15_optimizer = ...

q15_trained_model = train(
  q15_mlp, 
  q15_loss_fn, 
  q15_optimizer, 
  train_loader, 
  val_loader,
  epoch=15
)

In [ ]:
grader.check("Exercise 1-5")

#### 練習問題 1-6 | ReLU の動作確認

#### Exercise 1-6 | Confirming ReLU Functionality

$ReLU(x)$ について、`q16_x1 = -1`, `q16_x2 = 1` をそれぞれ $ReLU(x)$ の入力としたときの値 `q16_y1`, `q16_y2`と、勾配 `q16_g1`, `q16_g2` を計算せよ。ただし、ある torch.tensor `x` について、 `x.relu()` とすることで計算ができる。また、勾配は `backward()` を利用して計算すること。

For $ReLU(x)$, calculate the values `q16_y1` and `q16_y2` and the gradients `q16_g1` and `q16_g2` when `q16_x1 = -1` and `q16_x2 = 1` are used as inputs to $ReLU(x)$. For a given torch.tensor `x`, the calculation can be performed using `x.relu()`. Also, calculate the gradients using `backward()`.

In [ ]:

q16_x1=torch.tensor(-1.0, requires_grad=True)
q16_x2=torch.tensor(1.0, requires_grad=True)

q16_y1 = ...
q16_g1 = ...
q16_y2 = ...
q16_g2 = ...

print(q16_y1,q16_g1)
print(q16_y2,q16_g2)

In [ ]:
grader.check("Exercise 1-6")

#### 練習問題 1-7 | Dropout の動作確認

#### Exercise 1-7 | Confirming Dropout Functionality

2件の5次元ベクトル torch.tensor `q17_in` （`q17_in.shape = torch.Size([2, 5])`となるはずだ）を任意の値で準備し、`torch.nn.Dropout()` に入力したときの出力 `q17_out` を求めよ。

Prepare two 5-dimensional vectors `q17_in` (which should have `q17_in.shape = torch.Size([2, 5])`) with arbitrary values, and calculate the output `q17_out` when inputting it into `torch.nn.Dropout()`.

In [ ]:
torch.manual_seed(0)
q17_in = ...
q17_dropout = torch.nn.Dropout()
q17_out = ...

In [ ]:
grader.check("Exercise 1-7")

#### 練習問題 1-8 | 予測モデルの汎化性能評価

#### Exercise 1-8 | Evaluating Generalization Performance of the Prediction Model

以下に与えられるニューラルネットワーク `q18_nn` について訓練を行い、テストデータに対する正解率 `q18_test_acc` を `evaluate_model()` 関数を用いて求めよ。ただし、損失関数は `torch.nn.CrossEntropyLoss()` 、最適化手法は `torch.optim.SGD()` を用い、学習率は `0.05` とすること。

Train the given neural network `q18_nn` and use the `evaluate_model()` function to calculate the accuracy `q18_test_acc` on the test data. Use the loss function `torch.nn.CrossEntropyLoss()` and the optimization method `torch.optim.SGD()` with a learning rate of `0.05`.

In [ ]:
torch.manual_seed(0)
q18_nn = torch.nn.Sequential(
  torch.nn.Linear(8*8, 32),
  torch.nn.ReLU(),
  torch.nn.Dropout(0.5),
  torch.nn.Linear(32, 10),
)
 

In [ ]:
q18_loss_fn = ...
q18_optimizer = ...

In [ ]:
torch.manual_seed(0)
q18_train=train(
    q18_nn,
    q18_loss_fn,
    q18_optimizer,
    train_loader,
    val_loader
)

_,q18_test_acc = evaluate_model(q18_train,q18_loss_fn,test_loader)


In [ ]:
grader.check("Exercise 1-8")